In [1]:
import warnings
warnings.filterwarnings('ignore')
from category_encoders import LeaveOneOutEncoder,WOEEncoder
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(palette = "Dark2")
my_colors = [(0.10588235294117647, 0.6196078431372549, 0.4666666666666667),
  (0.8509803921568627, 0.37254901960784315, 0.00784313725490196)]
pd.set_option('display.max_columns', None)
from itertools import chain, combinations
import math

In [2]:
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_absolute_error
from catboost import CatBoostClassifier, Pool, CatBoostRegressor
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
import tensorflow as tensorflow
from lightgbm import LGBMClassifier
from sklearn.preprocessing import StandardScaler

In [3]:
df = pd.read_csv("df_97_columns.csv")
df

,order_item_id,order_date,delivery_date,item_id,size,item_color,brand_id,item_price,user_id,user_title,user_dob,user_state,user_reg_date,return,delivery_time,order_id,user_age,user_reg_age,order_weekday,delivery_weekday,order_month,delivery_month,order_day,delivery_day,order_week,delivery_week,order_item_count,order_sum,average_item_price_order,order_number_same_item_id,order_number_different_item_id,order_number_same_size,order_number_different_size,order_number_same_item_color,order_number_different_item_color,order_number_same_brand_id,order_number_different_brand_id,order_number_same_item_id_size,order_number_different_item_id_size,order_number_same_item_id_item_color,order_number_different_item_id_item_color,order_number_same_size_item_color,order_number_different_size_item_color,order_number_same_size_brand_id,order_number_different_size_brand_id,order_number_same_item_color_brand_id,order_number_different_item_color_brand_id,order_number_same_item_id_size_item_color,order_number_different_item_id_size_item_color,order_number_same_size_item_color_brand_id,order_number_different_size_item_color_brand_id,order_item_id_nunique,order_size_nunique,order_brand_id_nunique,order_item_color_nunique,order_item_id_color_nunique,order_item_id_size_nunique,order_brand_id_color_nunique,order_brand_id_size_nunique,order_brand_id_item_id_nunique,item_price_item_id_mean,item_price_item_id_std,item_price_item_id_min,item_price_item_id_max,item_price_item_id_sum,item_price_item_id_count,item_price_item_id_median,item_price_item_id_mad,item_price_user_id_mean,item_price_user_id_std,item_price_user_id_min,item_price_user_id_max,item_price_user_id_sum,item_price_user_id_count,item_price_user_id_median,item_price_user_id_mad,item_price_brand_id_mean,item_price_brand_id_std,item_price_brand_id_min,item_price_brand_id_max,item_price_brand_id_sum,item_price_brand_id_count,item_price_brand_id_median,item_price_brand_id_mad,mode_item_id,mode_size,mode_brand_id,mode_item_color,price-item_price_item_id_mean,price-item_price_item_id_min,price-item_price_item_id_max,price-item_price_user_id_mean,price-item_price_user_id_min,price-item_price_user_id_max,price-item_price_brand_id_mean,price-item_price_brand_id_min,price-item_price_brand_id_max
0,1,2012-04-01,2012-04-03,186,s,denim,25,69.90,794,Mrs,1965-01-06,Bad-Wue,2011-04-25,0.0,2,2012-04-01_794,47,342,6,1,4,4,1,3,13,14,3,209.8,69.93,1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,2,2,2,2,3,1,1,1,1,1,69.47,3.41,39.90,69.90,21606.68,311,69.90,0.83,62.65,24.51,0.0,89.9,689.1,11,69.9,16.52,55.23,13.65,24.9,79.90,282536.19,5116,59.90,11.66,71,s,20,white,-0.43,-30.0,0.0,-7.25,-69.90,20.00,-14.67,-45.00,10.0
1,2,2012-04-01,2012-04-03,71,unsized,ocher,21,69.95,794,Mrs,1965-01-06,Bad-Wue,2011-04-25,1.0,2,2012-04-01_794,47,342,6,1,4,4,1,3,13,14,3,209.8,69.93,2,1,2,1,1,2,2,1,2,1,1,2,1,2,2,1,1,2,1,2,1,2,2,2,2,3,2,1,2,1,1,57.95,13.28,34.95,69.95,59108.70,1020,69.95,12.35,62.65,24.51,0.0,89.9,689.1,11,69.9,16.52,74.54,30.32,24.9,179.95,478991.63,6426,69.95,22.66,71,s,20,white,-12.00,-35.0,0.0,-7.30,-69.95,19.95,4.59,-45.05,110.0
2,3,2012-04-01,2012-04-03,71,unsized,curry,21,69.95,794,Mrs,1965-01-06,Bad-Wue,2011-04-25,1.0,2,2012-04-01_794,47,342,6,1,4,4,1,3,13,14,3,209.8,69.93,2,1,2,1,1,2,2,1,2,1,1,2,1,2,2,1,1,2,1,2,1,2,2,2,2,3,2,1,2,1,1,57.95,13.28,34.95,69.95,59108.70,1020,69.95,12.35,62.65,24.51,0.0,89.9,689.1,11,69.9,16.52,74.54,30.32,24.9,179.95,478991.63,6426,69.95,22.66,71,s,20,white,-12.00,-35.0,0.0,-7.30,-69.95,19.95,4.59,-45.05,110.0
3,4,2012-04-02,2012-04-06,22,s,green,14,39.90,808,Mrs,1959-11-09,Saxony,2012-01-04,0.0,4,2012-04-02_808,52,89,0,4,4,4,2,6,14,14,1,39.9,39.90,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,1,1,1,1,1,1,1,1,31.10,8.68,19.90,39.90,68669.20,2208,29.90,8.41,39.90,0.00,39.9,39.9,39.9,1,39.9,0.00,55.16,19.34,0.0,89.90,629751.00,11417,49.90,16.21,22,s,14,green,-8.80,-20.0,0.0,0.00,0.00,0.00,15.26,-39.90,50.0
4,5,2012-04-02,2012-04-06,151,s,black,53,29.90,825,Mrs,1964-07-11,S-Holstein,2011-02-

In [3]:
# Create a class for all the models
class Model_class(object):
    def __init__(self, df:dict):
        self.df = df
        
        
    def split_data(self, historic:bool)->tuple:
        """
        Split the data into train and test sets, depending on whether the encdoder needs to be fited on the full data or not.

        Parameters
        ----------
        historic : bool
            False if the encoder needs to be fited on the Months April to February and to transform March
            True if the encoder needs to be fited on the Months April to December and to transform January to March

        Returns
        -------
        tuple
            df_train : dict
                Dataframe with training data.
            df_test : dict
                Dataframe with testing data.
        """
        # Get list of the months to train, remove months on which to test        
        if historic:
            months_to_train = list(range(1,13))
            # remove months 12,1,2,3 from the train set
            months_to_train = [i for i in months_to_train if i not in [12,1,2,3]]
        else:
            months_to_train = list(range(1,13))
            months_to_train.remove(3)
        k = max(self.df["order_item_id"])-1
        # Split into train and test. "~" in front of a variable means "not"
        df_train = self.df.loc[:k][self.df.loc[:k,"order_month"].isin(months_to_train)]
        df_test = self.df.loc[:k][~self.df.loc[:k,"order_month"].isin(months_to_train)]
        # Get the validation set
        # df_valid = self.df.iloc[k+1:, :]
        # Drop unnecessary columns
        columns_to_drop = ["order_date", "delivery_date", "user_dob", "user_reg_date", "order_id","order_item_id"]
        real_class = pd.read_csv("orders_realclass.txt", delimiter=";")
        df_valid = self.df.iloc[k+1:, :]
        df_valid.reset_index(drop=True,inplace=True)
        df_valid["return"] = real_class["returnShipment"]
        
        df_train.drop(columns_to_drop, axis=1, inplace=True)
        df_test.drop(columns_to_drop, axis=1, inplace=True)
        df_valid.drop(columns_to_drop, axis=1, inplace=True)
        return df_train, df_test, df_valid
    
    def LOE_Encoder(self, df_train:dict, df_test:dict, columns:list ,sig:float)->tuple:
        """
        Leave One Out Encoder to calculate the response variable for each category.

        Parameters
        ----------
        df_train : dict
            Dataframe with training data.
        df_test : dict
            Dataframe with testing data.
        columns : list
            Categorical columns to encode.
        sig : float
            Random noise added to the response variable.

        Returns
        -------
        tuple of the encoded dataframes and the encoder object.
        """              
        encoder = LeaveOneOutEncoder(cols=columns, return_df=True,sigma=sig)
        df_encode_train = encoder.fit_transform(df_train.drop(["return"],axis=1),df_train[["return"]]).round(3)
        df_encode_test = encoder.transform(df_test.drop(["return"],axis=1)).round(3)
        df_encode_train , df_encode_test = df_encode_train.join(df_train[["return"]]), df_encode_test.join(df_test[["return"]])
        return df_encode_train, df_encode_test, encoder
    
    def WOE_Encoder(self,df_train,df_test,columns,sig):
        encoder = WOEEncoder(cols=columns, return_df=True,sigma=sig, verbose=True,regularization=1)
        df_encode_train = encoder.fit_transform(df_train.drop(["return"],axis=1),df_train[["return"]])
        df_encode_test = encoder.transform(df_test.drop(["return"],axis=1))
        df_encode_train , df_encode_test = df_encode_train.join(df_train[["return"]]), df_encode_test.join(df_test[["return"]])
        return df_encode_train, df_encode_test, encoder
    
    def neural_network(self, df_train:dict, df_test:dict, n_layers:int, n_nodes:int, dropout:list, activation:str, optimizer:str, loss:str, metrics:list, epochs:int, batch_size:int,verbose:int)->object:
        """
        Neural network model to predict whether an item will be returned or not.

        Parameters
        ----------
        df_train : dict
            Dataframe with training data.
        df_test : dict
            Dataframe with testing data.
        n_layers : int
            Number of layers in the neural network.
        n_nodes : int
            Number of nodes in each layer.
        dropout : list
            List of dropout rates for each layer.
        activation : str
            Activation function for each layer except the last one.
        optimizer : str
            Optimizer for the neural network.
        loss : str
            Loss function for the neural network.
        metrics : str
            List of metrics for the neural network.
        epochs : int
            Number of epochs for the neural network.
        batch_size : int
            Size of the batch
        verbose : int
            Whether to print the progress of the neural network

        Returns
        -------
        model: object
            The Neural network model
        Y_pred: array
            Array of floats with the predictions of the neural network
        mae: float
            Mean absolute error on the testing set
        """         
        X_train, Y_train, X_test, Y_test = self.XY_split(df_train, df_test)
        X_train = StandardScaler().fit_transform(X_train)
        X_test = StandardScaler().fit_transform(X_test)
        
        model = Sequential()
        for i in range(n_layers):
            if i == 0:
                model.add(Dense(n_nodes[i], input_dim=X_train.shape[1], activation=activation))
            else:
                model.add(Dense(n_nodes[i], activation=activation))
            model.add(Dropout(dropout[i]))
        model.add(Dense(1, activation='sigmoid'))
        model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
        model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,verbose=verbose,validation_data=(X_test, Y_test))
        Y_pred = model.predict(X_test).round(4)
        mae = mean_absolute_error(Y_test, Y_pred)
        return model, Y_pred, mae,
    
    def xgboost(self, df_train:dict, df_test:dict, params:dict, verbose:int):
        X_train, Y_train, X_test, Y_test = self.XY_split(df_train, df_test)
        model = XGBClassifier(**params)
        model.fit(X_train, Y_train, eval_metric='mae', eval_set=[(X_test, Y_test)],early_stopping_rounds = 20,verbose = verbose)
        Y_pred = model.predict(X_test)
        Y_pred_prob = model.predict_proba(X_test)[:,1].round(4)
        mae = mean_absolute_error(Y_test, Y_pred)
        return model, Y_pred, mae, Y_pred_prob
    
    def catboost(self, df_train:dict, df_test:dict, params:dict, verbose:bool):
        X_train, Y_train, X_test, Y_test = self.XY_split(df_train, df_test)
        model = CatBoostClassifier(**params)
        model.fit(X_train, Y_train, eval_set=(X_test, Y_test), use_best_model=True, verbose=verbose,early_stopping_rounds = 20)
        Y_pred = model.predict(X_test)
        Y_pred_prob = model.predict_proba(X_test)[:,1].round(4)
        mae = mean_absolute_error(Y_test, Y_pred)
        return model, Y_pred, mae, Y_pred_prob
    
    def lightgmb(self, df_train:dict, df_test:dict, params:dict, verbose:int):
        X_train, Y_train, X_test, Y_test = self.XY_split(df_train, df_test)
        model = LGBMClassifier(**params)
        model.fit(X_train, Y_train, verbose=verbose)
        Y_pred_prob = model.predict_proba(X_test)[:,1].round(4)
        Y_pred = model.predict(X_test)
        mae = mean_absolute_error(Y_test, Y_pred)
        return model, Y_pred, mae, Y_pred_prob
    
    def combine_models(self, prob_dict,df_test:dict):
        prob_list = list(prob_dict.keys())
        best_error = 1
        for j in range(len(prob_dict)):
            for k in range(j+1,len(prob_dict)):
                for i in range(1,101):
                    first_term = prob_dict[prob_list[j]]*(i/100)
                    second_term = prob_dict[prob_list[k]]*((100-i)/100)
                    sum = first_term + second_term
                    error = mean_absolute_error(df_test["return"], sum.round())
                    if error < best_error:
                        best_error = error
                        best_i = i
                        best_model1 = prob_list[j] + "_" + str(i/100)
                        best_model2 = prob_list[k] + "_" + str((100-i)/100)
                        best_combination = best_model1 + "/" + best_model2
        return best_combination, best_error
    
    def XY_split(self, df_train:dict, df_test:dict):
        X_train, Y_train = df_train.drop(["return"],axis=1), df_train["return"]
        X_test, Y_test = df_test.drop(["return"],axis=1), df_test["return"]
        return X_train, Y_train, X_test, Y_test
    
    
    def get_feature_importance(model:object,df:dict)->list:
        """
        Get feature importance from the tree-based models.

        Parameters
        ----------
        model : object
            Tree model from sklearn.
        df  : dict
            Tree model from sklearn

        Returns
        -------
        list
            List of the features with their corresponding feature importance.
        """    
        return list(sorted(zip(df.columns.drop("return"), model.feature_importances_), key=lambda xx: xx[1], reverse=True))
        
    

In [7]:
Model = Model_class(df)

In [11]:
low_cardinality_cat_cols = ["size","item_color","user_title","user_state"]
high_cardinality_cat_cols = ["item_id","brand_id","user_id"]
cat_cols = low_cardinality_cat_cols + high_cardinality_cat_cols
cat_cols += ["mode_item_id","mode_size","mode_brand_id","mode_item_color",]

In [105]:
df_train,df_test, df_valid = Model.split_data(False)

In [29]:
df_train_encode,df_test_encode, encoder = Model.LOE_Encoder(df_train,df_test,cat_cols,0.2)

In [32]:
neural_network_params = {"n_layers":2, "n_nodes":[64,64], "dropout":[0.1,0.1], "activation":"relu", "optimizer":"adam", "loss":"mae", "metrics":["mae"], "epochs":15, "batch_size":256, "verbose":1}
neural_network_model, nn_Y_pred, nn_mae = Model.neural_network(df_train_encode, df_test_encode, **neural_network_params)
print("Neural Network MAE:",nn_mae)

Epoch 1/15
1608/1608 [==============================] - 4s 2ms/step - loss: 0.3386 - mae: 0.3386 - val_loss: 0.3623 - val_mae: 0.3623
Epoch 2/15
1608/1608 [==============================] - 3s 2ms/step - loss: 0.3271 - mae: 0.3271 - val_loss: 0.3640 - val_mae: 0.3640
Epoch 3/15
1608/1608 [==============================] - 4s 2ms/step - loss: 0.3246 - mae: 0.3246 - val_loss: 0.3648 - val_mae: 0.3648
Epoch 4/15
1608/1608 [==============================] - 4s 2ms/step - loss: 0.3241 - mae: 0.3241 - val_loss: 0.3635 - val_mae: 0.3635
Epoch 5/15
1608/1608 [==============================] - 3s 2ms/step - loss: 0.3236 - mae: 0.3236 - val_loss: 0.3620 - val_mae: 0.3620
Epoch 6/15
1608/1608 [==============================] - 3s 2ms/step - loss: 0.3229 - mae: 0.3229 - val_loss: 0.3626 - val_mae: 0.3626
Epoch 7/15
1608/1608 [==============================] - 3s 2ms/step - loss: 0.3222 - mae: 0.3222 - val_loss: 0.3613 - val_mae: 0.3613
Epoch 8/15
1608/1608 [==============================] - 3s 2ms

In [15]:
xgb_params = { "max_depth":9, "learning_rate":0.16, "n_estimators":50, "objective":"binary:logistic","random_state":42}
xgb_model, xgb_Y_pred, xgb_mae, xgb_Y_pred_prob = Model.xgboost(df_train_encode, df_test_encode, xgb_params, 1)
print("XGB MAE",xgb_mae)

In [37]:
# list_of_errors = []
# for i in range(1,101):
#     xgb_Y_pred_prob_threshold = xgb_Y_pred_prob.copy()
#     xgb_Y_pred_prob_threshold[xgb_Y_pred_prob_threshold>=i/100] = 1
#     xgb_Y_pred_prob_threshold[xgb_Y_pred_prob_threshold<i/100] = 0
#     # mean_absolute_error(df_test["return"],xgb_Y_pred_prob_threshold)
#     list_of_errors.append(mean_absolute_error(df_test["return"],xgb_Y_pred_prob_threshold))

In [31]:
catboost_params = { "depth":9, "random_seed":42, "iterations":100, "loss_function":"Logloss", "verbose":1,"learning_rate":0.15}
catboost_model, catboost_Y_pred, catboost_mae, catboost_Y_pred_prob = Model.catboost(df_train_encode, df_test_encode, catboost_params, 1)
print("Catboost MAE",catboost_mae)

0:	learn: 0.6629709	test: 0.6723072	best: 0.6723072 (0)	total: 225ms	remaining: 22.3s
1:	learn: 0.6432539	test: 0.6586959	best: 0.6586959 (1)	total: 399ms	remaining: 19.5s
2:	learn: 0.6282367	test: 0.6496332	best: 0.6496332 (2)	total: 584ms	remaining: 18.9s
3:	learn: 0.6172558	test: 0.6428240	best: 0.6428240 (3)	total: 761ms	remaining: 18.3s
4:	learn: 0.6075434	test: 0.6342311	best: 0.6342311 (4)	total: 968ms	remaining: 18.4s
5:	learn: 0.6015414	test: 0.6307368	best: 0.6307368 (5)	total: 1.14s	remaining: 17.9s
6:	learn: 0.5953330	test: 0.6263933	best: 0.6263933 (6)	total: 1.33s	remaining: 17.7s
7:	learn: 0.5918784	test: 0.6243011	best: 0.6243011 (7)	total: 1.5s	remaining: 17.3s
8:	learn: 0.5881345	test: 0.6222308	best: 0.6222308 (8)	total: 1.69s	remaining: 17.1s
9:	learn: 0.5851364	test: 0.6199034	best: 0.6199034 (9)	total: 1.88s	remaining: 16.9s
10:	learn: 0.5830651	test: 0.6186545	best: 0.6186545 (10)	total: 2.05s	remaining: 16.6s
11:	learn: 0.5810257	test: 0.6179720	best: 0.6179720 

In [30]:
lightgbm_params = { "learning_rate":0.15, "n_estimators":50, "objective":"binary","random_state":42}
lightgbm_model, lightgbm_Y_pred, lightgbm_mae, lightgbm_Y_pred_prob = Model.lightgmb(df_train_encode, df_test_encode, lightgbm_params, 1)
print("Lightgbm MAE",lightgbm_mae)

Lightgbm MAE 0.33417568848175055


In [75]:
# function to determine the best model through calculating the threshold to combine the predictions of the two models
prob_dict = {"xgb":xgb_Y_pred_prob, "catboost":catboost_Y_pred_prob, "lightgbm":lightgbm_Y_pred_prob, "neural_network":nn_Y_pred[:,0]}
def combine_models(prob_dict):
    prob_list = list(prob_dict.keys())
    error_best = 1
    for j in range(len(prob_dict)):
        for k in range(j+1,len(prob_dict)):
            for i in range(1,101):
                first_term = prob_dict[prob_list[j]]*(i/100)
                second_term = prob_dict[prob_list[k]]*((100-i)/100)
                sum = first_term + second_term
                error = mean_absolute_error(df_test["return"], sum.round())
                if error < error_best:
                    error_best = error
                    best_i = i
                    best_model1 = prob_list[j] + "_" + str(i/100)
                    best_model2 = prob_list[k] + "_" + str((100-i)/100)
                    best_combination = best_model1 + "/" + best_model2
    return error_best, best_combination

In [92]:
prob_dict = {"xgb":xgb_Y_pred_prob, "catboost":catboost_Y_pred_prob, "lightgbm":lightgbm_Y_pred_prob, "neural_network":nn_Y_pred[:,0]}
best_combination, best_combination_mae = Model.combine_models(prob_dict,df_test_encode)

Best Combination xgb_0.6/catboost_0.4


In [93]:
MAE_dict = {"Neural Network":nn_mae,"XGB":xgb_mae,"Catboost":catboost_mae,"Lightgbm":lightgbm_mae,str(best_combination):best_combination_mae}
MAE_df = pd.DataFrame(MAE_dict,index=["MAE_test_set"])
MAE_df.sort_values(by="MAE_test_set",ascending=True,inplace=True,axis=1)
MAE_df = round(MAE_df.transpose(),4) 
MAE_df

,MAE_test_set
xgb_0.6/catboost_0.4,0.3305
Catboost,0.3317
Lightgbm,0.3342
XGB,0.3343
Neural Network,0.3585


In [107]:
# concat df_train and df_test
df_train_conc = pd.concat([df_train,df_test],axis=0)

In [109]:
df_train_conc_encode, df_valid_encode, encoder = Model.LOE_Encoder(df_train_conc,df_valid,cat_cols,0.2)

In [112]:
neural_network_val, nn_Y_pred_val, nn_mae_val = Model.neural_network(df_train_conc_encode, df_valid_encode, **neural_network_params)
nn_mae_val

Epoch 1/15
1880/1880 [==============================] - 3s 2ms/step - loss: 0.3364 - mae: 0.3364 - val_loss: 0.3545 - val_mae: 0.3545
Epoch 2/15
1880/1880 [==============================] - 3s 2ms/step - loss: 0.3262 - mae: 0.3262 - val_loss: 0.3540 - val_mae: 0.3540
Epoch 3/15
1880/1880 [==============================] - 3s 2ms/step - loss: 0.3241 - mae: 0.3241 - val_loss: 0.3537 - val_mae: 0.3537s: 0.3243 - mae: 0. - ETA: 0s - loss: 0.3240 - mae: 0.3
Epoch 4/15
1880/1880 [==============================] - 5s 3ms/step - loss: 0.3237 - mae: 0.3237 - val_loss: 0.3544 - val_mae: 0.3544
Epoch 5/15
1880/1880 [==============================] - 5s 2ms/step - loss: 0.3223 - mae: 0.3223 - val_loss: 0.3479 - val_mae: 0.3479
Epoch 6/15
1880/1880 [==============================] - 3s 2ms/step - loss: 0.3207 - mae: 0.3207 - val_loss: 0.3479 - val_mae: 0.3479
Epoch 7/15
1880/1880 [==============================] - 3s 2ms/step - loss: 0.3201 - mae: 0.3201 - val_loss: 0.3481 - val_mae: 0.3481
Epoch 8

0.33952204361467275

In [ ]:
xgb_params = { "max_depth":9, "learning_rate":0.16, "n_estimators":50, "objective":"binary:logistic","random_state":42}
xgb_model_val, xgb_Y_pred_val, xgb_mae_val, xgb_Y_pred_prob_val = Model.xgboost(df_train_conc_encode, df_valid_encode, xgb_params, 1)

In [132]:
print("XGB MAE",xgb_mae_val)

XGB MAE 0.3175645992252087


In [116]:
catboost_model_val, catboost_Y_pred_val, catboost_mae_val, catboost_Y_pred_prob_val = Model.catboost(df_train_conc_encode, df_valid_encode, catboost_params, 0)
print("Catboost MAE",catboost_mae_val)

Catboost MAE 0.32155836894444667


In [117]:
lightgbm_model_val, lightgbm_Y_pred_val, lightgbm_mae_val, lightgbm_Y_pred_prob_val = Model.lightgmb(df_train_conc_encode, df_valid_encode, lightgbm_params, 0)
print("Lightgbm MAE",lightgbm_mae_val)

Lightgbm MAE 0.3201605495427134


In [126]:
combined_prob = xgb_Y_pred_prob_val*(0.6) + catboost_Y_pred_prob_val*(0.4)
combined_prob = combined_prob.round()
combined_error = mean_absolute_error(df_valid["return"],combined_prob).round(5)

In [128]:
total_error = 50078*xgb_mae_val
total_error

15903.0

In [9]:
df_train_historical, df_test_historical, df_valid = Model.split_data(True)

In [12]:
df_train_historical_encode, df_test_historical_encode, encoder = Model.WOE_Encoder(df_train_historical,df_test_historical,cat_cols,0.05)

In [13]:
months_to_train = [12,1,2]
df_test_historical_encode_train = df_test_historical_encode[df_test_historical_encode["order_month"].isin(months_to_train)]
df_test_historical_encode_test = df_test_historical_encode[~df_test_historical_encode["order_month"].isin(months_to_train)]

In [22]:
xgb_params = { "max_depth":9, "learning_rate":0.16, "n_estimators":150, "objective":"binary:logistic","random_state":42}
xgboost_model_historical, xgb_Y_pred_historical, xgb_mae_historical, xgb_Y_pred_prob_historical = Model.xgboost(df_test_historical_encode_train, df_test_historical_encode_test, xgb_params, 1)
print("XGB MAE",xgb_mae_historical)

[0]	validation_0-mae:0.49014
[1]	validation_0-mae:0.48140
[2]	validation_0-mae:0.47436
[3]	validation_0-mae:0.46856
[4]	validation_0-mae:0.46244
[5]	validation_0-mae:0.45726
[6]	validation_0-mae:0.45384
[7]	validation_0-mae:0.45042
[8]	validation_0-mae:0.44746
[9]	validation_0-mae:0.44405
[10]	validation_0-mae:0.44149
[11]	validation_0-mae:0.43895
[12]	validation_0-mae:0.43694
[13]	validation_0-mae:0.43486
[14]	validation_0-mae:0.43343
[15]	validation_0-mae:0.43219
[16]	validation_0-mae:0.43128
[17]	validation_0-mae:0.43062
[18]	validation_0-mae:0.42941
[19]	validation_0-mae:0.42808
[20]	validation_0-mae:0.42740
[21]	validation_0-mae:0.42679
[22]	validation_0-mae:0.42622
[23]	validation_0-mae:0.42555
[24]	validation_0-mae:0.42516
[25]	validation_0-mae:0.42439
[26]	validation_0-mae:0.42348
[27]	validation_0-mae:0.42299
[28]	validation_0-mae:0.42257
[29]	validation_0-mae:0.42236
[30]	validation_0-mae:0.42207
[31]	validation_0-mae:0.42184
[32]	validation_0-mae:0.42169
[33]	validation_0-ma

In [20]:
catboost_params = { "depth":9, "random_seed":42, "iterations":100, "loss_function":"Logloss", "verbose":1,"learning_rate":0.15}
catboost_model_val_historical, catboost_Y_pred_val_historical, catboost_mae_val_historical, catboost_Y_pred_prob_val_historical = Model.catboost(df_test_historical_encode_train, df_test_historical_encode_test, catboost_params, 1)
print("Catboost MAE",catboost_mae_val_historical)

0:	learn: 0.6701213	test: 0.6719566	best: 0.6719566 (0)	total: 85.9ms	remaining: 8.5s
1:	learn: 0.6550407	test: 0.6595996	best: 0.6595996 (1)	total: 163ms	remaining: 7.98s
2:	learn: 0.6438808	test: 0.6509606	best: 0.6509606 (2)	total: 249ms	remaining: 8.05s
3:	learn: 0.6352730	test: 0.6441494	best: 0.6441494 (3)	total: 332ms	remaining: 7.96s
4:	learn: 0.6289268	test: 0.6392159	best: 0.6392159 (4)	total: 415ms	remaining: 7.89s
5:	learn: 0.6240797	test: 0.6357934	best: 0.6357934 (5)	total: 500ms	remaining: 7.83s
6:	learn: 0.6206361	test: 0.6336700	best: 0.6336700 (6)	total: 597ms	remaining: 7.92s
7:	learn: 0.6158242	test: 0.6296205	best: 0.6296205 (7)	total: 687ms	remaining: 7.9s
8:	learn: 0.6131403	test: 0.6280576	best: 0.6280576 (8)	total: 762ms	remaining: 7.7s
9:	learn: 0.6111912	test: 0.6271252	best: 0.6271252 (9)	total: 858ms	remaining: 7.72s
10:	learn: 0.6082986	test: 0.6246954	best: 0.6246954 (10)	total: 941ms	remaining: 7.61s
11:	learn: 0.6065897	test: 0.6234829	best: 0.6234829 (

In [21]:
list(sorted(zip(df_test_historical_encode_train.columns.drop("return"), xgboost_model_historical.feature_importances_), key=lambda xx: xx[1], reverse=True))

[('item_price_item_id_max', 0.059217863),
 ('delivery_weekday', 0.05522489),
 ('order_item_id_size_nunique', 0.053468738),
 ('user_id', 0.05205973),
 ('item_price_item_id_mean', 0.049567867),
 ('item_price', 0.04147808),
 ('item_price_item_id_min', 0.034959912),
 ('order_sum', 0.023799375),
 ('order_number_same_item_id_item_color', 0.021620832),
 ('delivery_time', 0.019792031),
 ('brand_id', 0.01800419),
 ('item_id', 0.015281133),
 ('order_number_same_item_color_brand_id', 0.012338433),
 ('order_brand_id_size_nunique', 0.012132957),
 ('order_number_different_size_brand_id', 0.012038885),
 ('delivery_week', 0.011983824),
 ('order_item_count', 0.011448588),
 ('item_price_brand_id_median', 0.010690354),
 ('order_number_different_item_id_size_item_color', 0.010572391),
 ('order_number_different_size_item_color_brand_id', 0.010266334),
 ('item_price_user_id_sum', 0.00991724),
 ('order_item_id_nunique', 0.009706765),
 ('item_price_user_id_count', 0.0090891225),
 ('order_number_different_bran